# RAG Operations in Pixeltable

In this tutorial, we'll explore Pixeltable's flexible handling of RAG operations on unstructured text. In a traditional AI workflow, such operations might be implemented as a Python script that runs on a periodic schedule or in response to certain events. In Pixeltable, as with everything else, they are implemented as persistent table operations that update incrementally as new data becomes available. In our tutorial workflow, we'll chunk Wikipedia articles in various ways with a document splitter, then apply several kinds of embeddings to the chunks.

To run this tutorial, you'll need to have Pixeltable installed, along with the Huggingface `sentence_transformers` package.

## Set Up the Table Structure

We start by creating a Pixeltable workspace `rag_demo` (if it doesn't already exist) and setting up the table structure for our new workflow.

In [2]:
import pixeltable as pxt

# Create the Pixeltable workspace
pxt.create_dir('rag_demo', ignore_errors=True)

# Clean the database to ensure we're using fresh table instances
# (in case this demo has been run before)
pxt.drop_table('rag_demo.short_char_chunks', ignore_errors=True)
pxt.drop_table('rag_demo.short_chunks', ignore_errors=True)
pxt.drop_table('rag_demo.chunks', ignore_errors=True)
pxt.drop_table('rag_demo.sentences', ignore_errors=True)
pxt.drop_table('rag_demo.docs', ignore_errors=True)

Created directory `rag_demo`.


## Creating Tables and Views

Now we'll create the tables that represent our workflow, starting with a table to hold references to source documents. The table contains a single column `source_doc` whose elements have type `pxt.DocumentType`, representing a general document instance. In this tutorial, we'll be working with HTML documents, but Pixeltable supports a range of other document types, such as Markdown and PDF.

In [3]:
docs = pxt.create_table('rag_demo.docs', {'source_doc': pxt.DocumentType()})

Created table `docs`.


If we take a peek at the `docs` table, we see its very simple structure.

In [4]:
docs

Column Name,Type,Computed With
source_doc,document,


Next we create a view to represent chunks of our HTML documents. A Pixeltable view is a virtual table, which is dynamically derived from a source table by applying a transformation and/or selecting a subset of data. In this case, our view represents a one-to-many transformation from source documents into individual sentences. This is achieved using Pixeltable's built-in `DocumentSplitter` class.

Note that the `docs` table is currently empty, so creating this view doesn't actually *do* anything yet: it simply defines an operation that we want Pixeltable to execute when it sees new data.

In [4]:
from pixeltable.iterators.document import DocumentSplitter

sentences = pxt.create_view(
    'rag_demo.sentences',  # Name of the view
    docs,  # Table from which the view is derived
    iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'sentence',  # Chunk docs into sentences
        'metadata': 'title,headings,sourceline',
    })

Created view `sentences` with 0 rows, 0 exceptions.


Let's take a peek at the new `sentences` view. Run the following command:

In [5]:
sentences

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
headings,json,
sourceline,int,
source_doc,document,


We see that `sentences` inherits the `source_doc` column from `docs`, together with some new fields:
- `pos`: The position in the source document where the sentence appears.
-  `text`: The text of the sentence.
- `title`, `headings`, and `sourceline`: The metadata we requested when we set up the view.

## Data Ingestion

Ok, now it's time to insert some data into our workflow. A document in Pixeltable is just a URL; the following command inserts a single row into the `docs` table with the `source_doc` field set to the specified URL:

In [6]:
docs.insert(source_doc='https://en.wikipedia.org/wiki/Marc_Chagall')

Inserting rows into `docs`: 1 rows [00:00, 182.72 rows/s]
Inserting rows into `sentences`: 1465 rows [00:00, 3008.51 rows/s]

Inserted 1466 rows with 0 errors.


UpdateStatus(num_rows=1466, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

We can see that two things happened. First, a single row was inserted into `docs`, containing the URL representing our source document. Then, the view `sentences` was incrementally updated by applying the `DocumentSplitter` according to the definition of the view. This illustrates an important principle in Pixeltable: by default, anytime Pixeltable sees new data, the update is incrementally propagated to any downstream views or computed columns.

We can see the effect of the insertion with the `select` command. There's a single row in `docs`:

In [7]:
docs.select(docs.source_doc.fileurl).show()

source_doc_fileurl
https://en.wikipedia.org/wiki/Marc_Chagall


And here are the first 20 rows in `sentences`. The content of the article is broken into individual sentences, as expected.

In [8]:
sentences.select(sentences.text, sentences.headings).show(20)

text,headings
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية,{'1': 'Marc Chagall'}
Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी,{'1': 'Marc Chagall'}
Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย,{'1': 'Marc Chagall'}
Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש,{'1': 'Marc Chagall'}
"粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here.",{'1': 'Marc Chagall'}
"For other uses, see Chagall (disambiguation) .",{'1': 'Marc Chagall'}
"Marc Chagall Chagall, c. 1920",{'1': 'Marc Chagall'}
"Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire",{'1': 'Marc Chagall'}
[1] Died 28 March 1985 (1985-03-28) (aged 97),{'1': 'Marc Chagall'}


## Experimenting with Chunking

Of course, chunking into sentences isn't the only way to split a document. Perhaps we want to experiment with different chunking methodologies, in order to see which one performs best in a particular application. Pixeltable makes it easy to do this, by creating several views of the same source table. Here are a few examples. Notice that as each new view is created, it is initially populated from the data already in `docs`.

In [9]:
chunks = pxt.create_view(
    'rag_demo.chunks', docs, iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'paragraph,token_limit',
        'limit': 2048,
        'overlap': 0,
        'metadata': 'title,headings,sourceline',
    })

Inserting rows into `chunks`: 205 rows [00:00, 8721.47 rows/s]

Created view `chunks` with 205 rows, 0 exceptions.


In [10]:
short_chunks = pxt.create_view(
    'rag_demo.short_chunks', docs, iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'paragraph,token_limit',
        'limit': 72,
        'overlap': 0,
        'metadata': 'title,headings,sourceline',
    })

Inserting rows into `short_chunks`: 527 rows [00:00, 10806.26 rows/s]

Created view `short_chunks` with 527 rows, 0 exceptions.


In [11]:
short_char_chunks = pxt.create_view(
    'rag_demo.short_char_chunks', docs, iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'paragraph,char_limit',
        'limit': 72,
        'overlap': 0,
        'metadata': 'title,headings,sourceline',
    })

Inserting rows into `short_char_chunks`: 1759 rows [00:00, 9943.34 rows/s]

Created view `short_char_chunks` with 1759 rows, 0 exceptions.


In [22]:
chunks.select(chunks.text, chunks.headings).show(20)

text,headings
"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .",{'1': 'Marc Chagall'}
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire [1] Died 28 March 1985 (1985-03-28) (aged 97) Saint-Paul-de-Vence , France Nationality Russian, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Valentina (Vava) Brodsky ​ ​ ( m. 1952) ​ [3] Children 2 [4]",{'1': 'Marc Chagall'}
"Marc Chagall [a] (born Moishe Shagal ; 6 July [ O.S. 24 June] 1887 – 28 March 1985) was a Russian-French artist. [b] An early modernist , he was associated with the École de Paris as well as several major artistic styles and created works in a wide range of artistic formats, including painting, drawings, book illustrations, stained glass , stage sets, ceramics, tapestries and fine art prints.",{'1': 'Marc Chagall'}
"Chagall was born in 1887, into a Jewish family near Vitebsk , today in Belarus , but at that time in the Pale of Settlement of the Russian Empire. Before World War I , he travelled between Saint Petersburg , Paris , and Berlin . During that period, he created his own mixture and style of modern art, based on his ideas of Eastern European and Jewish folklore. He spent the wartime years in his native Belarus, becoming one of the country's most distinguished artists and a member of the modernist avant-garde , founding the Vitebsk Arts College . He later worked in and near Moscow in difficult conditions during hard times in Russia following the Bolshevik Revolution , before leaving again for Paris in 1923. During World War II , he escaped occupied France to the United States, where he lived in New York City for seven years before returning to France in 1948.",{'1': 'Marc Chagall'}
"Art critic Robert Hughes referred to Chagall as ""the quintessential Jewish artist of the twentieth century"". According to art historian Michael J. Lewis, Chagall was considered to be ""the last survivor of the first generation of European modernists"". For decades, he ""had also been respected as the world's pre-eminent Jewish artist"". [15] Using the medium of stained glass, he produced windows for the cathedrals of Reims and Metz as well as the Fraumünster in Zürich , windows for the UN and the Art Institute of Chicago and the Jerusalem Windows in Israel. He also did large-scale paintings, including part of the ceiling of the Paris Opéra . He experienced modernism's ""golden age"" in Paris, where ""he synthesized the art forms of Cubism , Symbolism , and Fauvism , and the influence of Fauvism gave rise to Surrealism "". Yet throughout these phases of his style ""he remained most emphatically a Jewish artist, whose work was one long dreamy reverie of life in his native village of Vitebsk."" [16] ""When Matisse dies"", Pablo Picasso remarked in the 1950s, ""Chagall will be the only painter left who understands what colour really is"". [17]",{'1': 'Marc Chagall'}
Early life and ed

In [23]:
short_chunks.select(short_chunks.text, short_chunks.headings).show(20)

text,headings
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡ,{'1': 'Marc Chagall'}
ортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한,{'1': 'Marc Chagall'}
국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lë,{'1': 'Marc Chagall'}
tzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemont,{'1': 'Marc Chagall'}
èis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi,{'1': 'Marc Chagall'}
"Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here",{'1': 'Marc Chagall'}
". For other uses, see Chagall (disambiguation) .",{'1': 'Marc Chagall'}
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire [1] Died 28 March 1985 (1985-03-28) (aged",{'1': 'Marc Chagall'}
"97) Saint-Paul-de-Vence , France Nationality Russian, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Valentina (Vava)",{'1': 'Marc Chagall'}


In [24]:
short_char_chunks.select(short_char_chunks.text, short_char_chunks.headings).show(20)

text,headings
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտ,{'1': 'Marc Chagall'}
ահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (т,{'1': 'Marc Chagall'}
арашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικ,{'1': 'Marc Chagall'}
ά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrv,{'1': 'Marc Chagall'}
atski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswa,{'1': 'Marc Chagall'}
hili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy,{'1': 'Marc Chagall'}
مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzb,{'1': 'Marc Chagall'}
ekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português R,{'1': 'Marc Chagall'}
omână Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina,{'1': 'Marc Chagall'}


Now let's add a few more documents to our workflow. Notice how all of the downstream views are updated incrementally, processing just the new documents as they are inserted.

In [15]:
urls = [
    'https://en.wikipedia.org/wiki/Pierre-Auguste_Renoir',
    'https://en.wikipedia.org/wiki/Henri_Matisse',
    'https://en.wikipedia.org/wiki/Marcel_Duchamp'
]
docs.insert({'source_doc': url} for url in urls)

Inserting rows into `docs`: 3 rows [00:00, 1907.37 rows/s]
Inserting rows into `sentences`: 2104 rows [00:03, 602.96 rows/s]
Inserting rows into `chunks`: 274 rows [00:00, 8737.80 rows/s]
Inserting rows into `short_chunks`: 802 rows [00:00, 10560.29 rows/s]
Inserting rows into `short_char_chunks`: 2622 rows [00:00, 5677.07 rows/s]

Inserted 5805 rows with 0 errors.


UpdateStatus(num_rows=5805, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

## Further Experiments

This is a good time to mention another important guiding principle of Pixeltable. The preceding examples all used the built-in `DocumentSplitter` class with various configurations. That's probably fine as a first cut or to prototype an application quickly, and it might be sufficient for some applications. But other applications might want to do more sophisticated kinds of chunking, implementing their own specialized logic or leveraging third-party tools. Pixeltable imposes no constraints on the AI or RAG operations a workflow uses: the iterator interface is highly general, and it's easy to implement new operations or adapt existing code or third-party tools into the Pixeltable workflow. Pixeltable manages *data* storage and workflow that is common to all such operations; above the data plane, it's fully general and extensible.

TODO: Example of a custom splitter?

## Adding embeddings

Next, let's look at how embeddings can be added seamlessly to existing Pixeltable workflows. To compute our embeddings, we'll use the Huggingface `sentence_transformer` package, running it over the `chunks` view that broke our documents up into larger paragraphs. Pixeltable has a built-in `sentence_transformer` adapter, and all we have to do is add a new column that leverages it. Pixeltable takes care of the rest, applying the new column to all existing data in the view.

In [16]:
from pixeltable.functions.huggingface import sentence_transformer, clip

minilm_model_id = 'paraphrase-MiniLM-L6-v2'

chunks.add_column(minilm_embedding=sentence_transformer(chunks.text, model_id=minilm_model_id))

Computing cells:   0%|                                                                                                                                                                                                                                                                    | 0/479 [00:00<?, ?cells/s]/Users/asiegel/Dropbox/workspace/pixeltable/pixeltable/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Computing cells: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

added 479 column values with 0 errors


UpdateStatus(num_rows=479, num_computed_values=479, num_excs=0, updated_cols=[], cols_with_excs=[])

The new column is a *computed column*: it is defined as a function on top of existing data and updated incrementally as new data are added to the workflow. Let's have a look at how the new column affected the `chunks` view.

In [17]:
chunks

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
headings,json,
sourceline,int,
minilm_embedding,"array((None,), dtype=FLOAT)","huggingface.sentence_transformer(text, model_id='paraphrase-MiniLM-L6-v2')"
source_doc,document,


In [18]:
chunks.head()

pos,text,title,headings,sourceline,minilm_embedding,source_doc
0,Marc Chagall - Wikipedia Jump to content Search Search,Marc Chagall - Wikipedia,{},0,"[-0.26239744, -0.11875608, -0.13270901, 0.048251476, 0.11987579, -0.0060523422, 0.32066357, 0.04835883, 0.29429552, -0.25400946, 0.6250121, 0.17804725, -0.0049092094, 0.26455665, -0.25857177, -0.04235531, 0.36528507, 0.23551288, 0.27595958, -0.3712053, 0.17573264, 0.0029876528, 0.23927844, -0.17485948, 0.1954067, -0.48891908, -0.7418267, -0.30720565, 0.23550902, -0.26159504, 0.5011105, -0.6918158, -0.25462586, -0.19169782, 0.5265674, 0.10684755, -0.3500413, -0.47890776, 0.27644473, 0.2624894, 0.055178, -0.1461069, -0.21482669, 0.7246923, -0.105297744, 0.5106913, -0.18174116, 0.45920599, 0.36721304, 0.51290447, -0.65757185, -0.4469795, -0.26267004, -0.44820127, 0.38971028, -0.1695206, -0.114350036, -0.07236255, 0.2645084, -0.5793037, 0.28047276, 0.091701195, -0.16436026, 0.10646367, -0.1461572, 0.19661109, 0.07363955, 0.21059886, 0.1799219, 0.1598077, -0.37105122, 0.25376227, -0.4467154, 0.6573318, -0.38872772, -0.2764637, -0.79460824, -0.08837392, -0.27318305, -0.21008545, 0.57023394, -0.3434785, 0.15152003, -0.30319378, -0.043730583, -0.3291485, 0.2555499, 0.060874756, 0.23776941, 0.29614958, 0.18648784, -0.35599723, 0.15256202, -0.084133916, -0.48064327, 0.30865645, 0.1097052, 0.11276893, 0.718605, -0.21019463, ...]",/Users/asiegel/.pixeltable/file_cache/a34a1838d0db4b6297e5c301481ee747_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
1,"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .",Marc Chagall - Wikipedia,{'1': 'Marc Chagall'},820,"[-0.1363125, 0.40063262, -0.53002965, -0.18143149, -0.45316985, -0.124922395, 0.7796174, -0.24712479, 0.12325851, -0.22153592, 0.28455305, -0.4370274, 0.23156813, 0.14490244, -0.1085123, -0.28869802, 0.21817636, 0.7690572, -0.31989712, -0.44312742, 0.059898064, -0.08027313, 0.12561844, 0.20749849, -0.04779504, 0.2481188, 0.31219974, -0.08558018, -0.22011802, 0.3522022, 0.58977985, 0.07881048, 0.19522715, -0.14123625, 0.35312974, 0.24841884, -0.13180904, 0.27224553, 0.28210613, 0.13397679, 0.10607662, -0.09232119, 0.09185555, 0.18311317, 0.12448643, -0.050988674, -0.17604467, 0.015647216, -0.05244761, -0.24283391, -0.42616737, -0.45576864, -0.35507995, -0.43146488, 0.30693346, -0.9420446, -0.15023254, 0.73326993, 0.0008648038, -0.36632615, -0.5619301, 0.21601507, -0.6227495, 0.43365502, -0.2858443, -0.33551332, -0.044290252, 0.3440103, -0.2007212, 0.051950164, -0.14124432, -0.12208969, 0.17209955, 0.24971402, -0.114884056, -0.36850527, 0.2503845, 0.2248823, -0.23860496, 0.014345463, 0.13038495, -0.005723685, -0.030419044, -0.29217696, 0.23628014, -0.0753976, -0.011316212, -0.09114735, 0.6540441, -0.9468982, 0.51743305, 0.29965192, 0.23870105, -0.06947729, 0.50110245, -0.08394459, 0.15647276, -0.33553198, 0.086335674, -0.11068067, ...]",/Users/asiegel/.pixeltable/file_cache/a34a1838d0db4b6297e5c301481ee747_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
2,"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N

Similarly, we might want to add a CLIP embedding to our workflow; once again, it's just another computed column:

In [19]:
clip_model_id = 'openai/clip-vit-base-patch32'

chunks.add_column(clip_embedding=clip(text=chunks.text, model_id=clip_model_id))

Computing cells: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 479/479 [00:30<00:00, 15.84cells/s]

added 479 column values with 0 errors


UpdateStatus(num_rows=479, num_computed_values=479, num_excs=0, updated_cols=[], cols_with_excs=[])

In [20]:
chunks

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
headings,json,
sourceline,int,
minilm_embedding,"array((None,), dtype=FLOAT)","huggingface.sentence_transformer(text, model_id='paraphrase-MiniLM-L6-v2')"
clip_embedding,"array((None,), dtype=FLOAT)","huggingface.clip(text=text, model_id='openai/clip-vit-base-patch32')"
source_doc,document,


In [21]:
chunks.head()

pos,text,title,headings,sourceline,minilm_embedding,clip_embedding,source_doc
0,Marc Chagall - Wikipedia Jump to content Search Search,Marc Chagall - Wikipedia,{},0,"[-0.26239744, -0.11875608, -0.13270901, 0.048251476, 0.11987579, -0.0060523422, 0.32066357, 0.04835883, 0.29429552, -0.25400946, 0.6250121, 0.17804725, -0.0049092094, 0.26455665, -0.25857177, -0.04235531, 0.36528507, 0.23551288, 0.27595958, -0.3712053, 0.17573264, 0.0029876528, 0.23927844, -0.17485948, 0.1954067, -0.48891908, -0.7418267, -0.30720565, 0.23550902, -0.26159504, 0.5011105, -0.6918158, -0.25462586, -0.19169782, 0.5265674, 0.10684755, -0.3500413, -0.47890776, 0.27644473, 0.2624894, 0.055178, -0.1461069, -0.21482669, 0.7246923, -0.105297744, 0.5106913, -0.18174116, 0.45920599, 0.36721304, 0.51290447, -0.65757185, -0.4469795, -0.26267004, -0.44820127, 0.38971028, -0.1695206, -0.114350036, -0.07236255, 0.2645084, -0.5793037, 0.28047276, 0.091701195, -0.16436026, 0.10646367, -0.1461572, 0.19661109, 0.07363955, 0.21059886, 0.1799219, 0.1598077, -0.37105122, 0.25376227, -0.4467154, 0.6573318, -0.38872772, -0.2764637, -0.79460824, -0.08837392, -0.27318305, -0.21008545, 0.57023394, -0.3434785, 0.15152003, -0.30319378, -0.043730583, -0.3291485, 0.2555499, 0.060874756, 0.23776941, 0.29614958, 0.18648784, -0.35599723, 0.15256202, -0.084133916, -0.48064327, 0.30865645, 0.1097052, 0.11276893, 0.718605, -0.21019463, ...]","[0.43900743, -0.20442066, -0.3703856, -0.24842092, 0.2525654, 0.10988422, 0.17879994, -0.196813, -0.21898538, 0.16312195, -0.11581904, 0.10922383, -0.3980483, 0.09145157, 0.10282475, -0.08496836, -0.26351994, 0.21744385, -0.3051855, -0.25689328, 0.3959951, 0.050230637, 0.069754474, 0.024919227, -0.23106074, 0.40560833, 0.055413835, 0.32912296, 0.308473, -0.5097272, -0.09413433, -0.11475259, 0.060904503, -0.4149298, 0.22001794, -0.18334071, -0.17864163, -0.31954357, 0.25738722, -0.22377047, 0.18910344, 0.026809722, -0.004293531, 0.40239182, 0.20990916, 0.097469255, -0.13026372, -0.22480412, -0.57115316, -0.08310877, 0.4732533, -0.18007854, 0.1250679, 0.086631775, 0.9078158, 0.2944157, 0.37376213, 0.0021959953, -0.45549828, 0.3129748, 0.1539275, 0.1920172, 0.4021812, 0.07109217, 0.1754788, -0.15966587, 0.23044436, 0.17929715, -0.062350556, -0.14394853, 0.08396445, -0.1978707, 0.15875788, 0.15464768, -0.12898067, -0.12406565, -0.4527824, 0.09551789, 0.0028743632, 0.68251455, -0.59881175, -0.33829093, -0.3487394, 0.075701036, -0.35363907, -0.31604585, -0.14599548, -0.5174625, -0.13107216, 0.020409035, -0.47953704, -0.080894835, -1.0206147, -0.15478382, 0.2360041, 0.29316625, -0.44745338, -0.07385804, -0.11532639, -0.1653951, ...]",/Users/asiegel/.pixeltable/file_cache/a34a1838d0db4b6297e5c301481ee747_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
1,"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .",Marc Chagall - Wikipedia,{'1': 'Marc Chagall'},820,"[-0.1363125, 0.40063262, -0.53002965, -0.18143149, -0.45316985, -0.124922395, 0.7796174, -0.24712479, 0.12325851, -0.22153592, 0.28455305, -0.4370274, 0.23156813, 0.14490244, -0.1085123, -0.28869802, 0.21817636,